In [1]:
#connect to nfldb and load gameStats dataframe
import nfldb
import pandas as pd
import urllib2
from lxml import etree
from bs4 import BeautifulSoup
from bs4 import Comment
from urllib import urlopen
import numpy as np
import time


db = nfldb.connect()
gameStats = pd.read_csv('gameStats_withFeats_v2.csv')

/Users/richardknoche/anaconda/envs/py27/lib/python2.7/site-packages/matplotlib/__init__.py:1035: UserWarning: Duplicate key in file "/Users/richardknoche/.matplotlib/matplotlibrc", line #2
  (fname, cnt))


Add a "season week" and "year" column to the data frame, for easy cross checking with the web crawling results

In [2]:
gameStats['Year']=gameStats['Season']+2008
gameStats['SeasonWeek']=gameStats['Week']-17*(gameStats['Season']-1)

gameStats['Expert Rank']=None
gameStats['Average Expert Rank']=None
gameStats['Best Expert Rank']=None
gameStats['Worst Expert Rank']=None
gameStats['Injury Status']=None

gameStats['PPR Expert Rank']=None
gameStats['PPR Average Expert Rank']=None
gameStats['PPR Best Expert Rank']=None
gameStats['PPR Worst Expert Rank']=None
gameStats['PPR Injury Status']=None

Open the chached fantasy pros web page with beautiful soup. Trying 2011 format first.  Note- there is no 2013 archive

In [3]:
year='2011'

#Season starts first week in September, ends at end of December
for month in ['09','10','11','12','01']:
    for day in range(3,31,3):
        for pos in ['wr','rb','te','qb']:
            web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/%s.php' % (year,str(month).zfill(2),str(day).zfill(2),pos)

            #Check Jan of the next year too
            if month=='01':   
                web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/%s.php' % (str(int(year)+1),str(month).zfill(2),str(day).zfill(2),pos)
            
            web_html = urlopen(web_add).read()
            soup = BeautifulSoup(web_html)

            #Check if unavailable issue
            #If it's there, retry in 10 seconds
            while 'Service Unavailable' in soup.text:
                print 'No Server Avaialble.  Trying Again in 10 seconds...'
                time.sleep(10)
                web_html = urlopen(web_add).read()
                soup = BeautifulSoup(web_html)

            #Get the week and year of the predictions
            title= soup.find("title")
            web_week=int(title.text.split(' ')[1])
            
            #catch week 17 leaking into the start of the year
            if (month=='09') and (int(web_week)>10):
                print "Warning: Web week %s found in month 09" % web_week

            comments=soup.find_all(string=lambda text:isinstance(text,Comment))
            web_year = comments[-1].split(' ')[11]
            web_month = comments[-1].split(' ')[9]
            if web_year != year:
                print "Warning: Web year and Season Year mismatch.  Web month is %s" % web_month
            
            #Get the player rankings and injury status
            table= soup.find("table", { "id" : "data"} )

            #Skip the first row of the table, since it is header info
            rows=table.findAll('tr')
            iterrows = iter(rows)
            next(iterrows)

            #Iterate over reaming rows
            for tr in iterrows:

                #Find all columns and exctract info
                cols = tr.findAll('td')  
                
                #Skip empty rows and free agents (cols[4].find('a') empty means free agent)
                if cols[3].find('a').contents and cols[4].find('a'):
                    team = cols[4].find('a').contents[0] #Uses JAC already        
                    player = cols[3].find('a').contents[0]   

                    #If the player isn't in our database, find the closest matching name
                    #It's okay if it comes back with bull, since the player + team + season + week
                    #combo will prevent it from being entered
                    if player not in gameStats['Player'].tolist():
                        player = str(nfldb.player_search(db,player)[0])

                    rank=cols[0].text
                    best_rank=cols[1].text
                    worst_rank=cols[2].text
                    if len(cols[6].contents)>1:
                        injury_status=cols[6].contents[1][0]
                    else:
                        injury_status='H' #H for healthy

                    #Use player,team,week,and year to fill in the information in our dataframe        
                    selector=( (gameStats.Player==player) & (gameStats.Team ==team) &\
                               (gameStats.SeasonWeek==int(web_week)) & (gameStats.Year==float(year)) )


                    #If there is a player,team,season,and week match.. fill in the data
                    if len(gameStats[selector])>0: 
                        gameStats.ix[selector, 'Expert Rank'] = rank
                        #No average rank available
                        gameStats.ix[selector, 'Best Expert Rank']=best_rank
                        gameStats.ix[selector, 'Worst Expert Rank']=worst_rank
                        gameStats.ix[selector, 'Injury Status']=injury_status
                        
        print 'Done with %s/%s' % (month,day)

/Users/richardknoche/anaconda/envs/py27/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Done with 09/3
Done with 09/6
Done with 09/9
Done with 09/12
Done with 09/15
Done with 09/18
Done with 09/21
Done with 09/24
Done with 09/27
Done with 09/30
Done with 10/3
Done with 10/6
Done with 10/9
Done with 10/12
Done with 10/15
Done with 10/18
Done with 10/21
Done with 10/24
Done with 10/27
Done with 10/30
Done with 11/3
Done with 11/6
Done with 11/9
Done with 11/12
Done with 11/15
Done with 11/18
Done with 11/21
Done with 11/24
Done with 11/27
Done with 11/30
Done with 12/3
Done with 12/6
Done with 12/9
Done with 12/12
Done with 12/15
Done with 12/18
Done with 12/21
Done with 12/24
Done with 12/27
Done with 12/30
Done with 01/3
Done with 01/6
Done with 01/9
Done with 01/12
Done with 01/15
Done with 01/18
Done with 01/21
Done with 01/24
Done with 01/27
Done with 01/30


In [4]:
gameStats.to_csv('gameStats_withExperts.csv')

In [5]:
#PPR version
year='2011'

#Season starts first week in September, ends at end of December
for month in ['09','10','11','12','01']:
    for day in range(3,31,3):
        for pos in ['wr','rb','te']:
            web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/ppr-%s.php' % (year,str(month).zfill(2),str(day).zfill(2),pos)

            #Check Jan of the next year too
            if month=='01':   
                web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/ppr-%s.php' % (str(int(year)+1),str(month).zfill(2),str(day).zfill(2),pos)
             
            web_html = urlopen(web_add).read()
            soup = BeautifulSoup(web_html)

            #Check if unavailable issue
            #If it's there, retry in 10 seconds
            while 'Service Unavailable' in soup.text:
                print 'No Server Avaialble.  Trying Again in 10 seconds...'
                time.sleep(10)
                web_html = urlopen(web_add).read()
                soup = BeautifulSoup(web_html)

            #Get the week and year of the predictions
            title= soup.find("title")
            web_week=int(title.text.split(' ')[1])

            #catch week 17 leaking into the start of the year
            if (month=='09') and (int(web_week)>10):
                print "Warning: Web week %s found in month 09" % web_week
            
            
            comments=soup.find_all(string=lambda text:isinstance(text,Comment))
            web_year = comments[-1].split(' ')[11]
            web_month = comments[-1].split(' ')[9]
            if web_year != year:
                print "Warning: Web year and Season Year mismatch.  Web month is %s" % web_month


            #Get the player rankings and injury status
            table= soup.find("table", { "id" : "data"} )

            #Skip the first row of the table, since it is header info
            rows=table.findAll('tr')
            iterrows = iter(rows)
            next(iterrows)

            #Iterate over reaming rows
            for tr in iterrows:

                #Find all columns and exctract info
                cols = tr.findAll('td')  
                
                #Skip empty rows and free agents (cols[4].find('a') empty means free agent)
                if cols[3].find('a').contents and cols[4].find('a'):
                    team = cols[4].find('a').contents[0] #Uses JAC already        
                    player = cols[3].find('a').contents[0]   

                    #If the player isn't in our database, find the closest matching name
                    #It's okay if it comes back with bull, since the player + team + season + week
                    #combo will prevent it from being entered
                    if player not in gameStats['Player'].tolist():
                        player = str(nfldb.player_search(db,player)[0])

                    rank=cols[0].text
                    best_rank=cols[1].text
                    worst_rank=cols[2].text
                    if len(cols[6].contents)>1:
                        injury_status=cols[6].contents[1][0]
                    else:
                        injury_status='H' #H for healthy

                    #Use player,team,week,and year to fill in the information in our dataframe        
                    selector=( (gameStats.Player==player) & (gameStats.Team ==team) &\
                               (gameStats.SeasonWeek==int(web_week)) & (gameStats.Year==float(year)) )


                    #If there is a player,team,season,and week match.. fill in the data
                    if len(gameStats[selector])>0: 
                        gameStats.ix[selector, 'PPR Expert Rank'] = rank
                        #No average rank available
                        gameStats.ix[selector, 'PPR Best Expert Rank']=best_rank
                        gameStats.ix[selector, 'PPR Worst Expert Rank']=worst_rank
                        gameStats.ix[selector, 'PPR Injury Status']=injury_status
                        
        print 'Done with %s/%s' % (month,day)

Done with 09/3
Done with 09/6
Done with 09/9
Done with 09/12
Done with 09/15
Done with 09/18
Done with 09/21
Done with 09/24
Done with 09/27
Done with 09/30
Done with 10/3
Done with 10/6
Done with 10/9
Done with 10/12
Done with 10/15
Done with 10/18
Done with 10/21
Done with 10/24
Done with 10/27
Done with 10/30
Done with 11/3
Done with 11/6
Done with 11/9
Done with 11/12
Done with 11/15
Done with 11/18
Done with 11/21
Done with 11/24
Done with 11/27
Done with 11/30
Done with 12/3
Done with 12/6
Done with 12/9
Done with 12/12
Done with 12/15
Done with 12/18
Done with 12/21
Done with 12/24
Done with 12/27
Done with 12/30
Done with 01/3
Done with 01/6
Done with 01/9
Done with 01/12
Done with 01/15
Done with 01/18
Done with 01/21
Done with 01/24
Done with 01/27
Done with 01/30


In [6]:
gameStats.to_csv('gameStats_withExperts.csv')

In [7]:
year='2012'

#Season starts first week in September, ends at end of December
for month in ['09','10','11','12','01']:
    for day in range(3,31,3):
        for pos in ['wr','rb','te','qb']:
            web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/%s.php' % (year,str(month).zfill(2),str(day).zfill(2),pos)

            #Check Jan of the next year too
            if month=='01':   
                web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/%s.php' % (str(int(year)+1),str(month).zfill(2),str(day).zfill(2),pos)
  
            web_html = urlopen(web_add).read()
            soup = BeautifulSoup(web_html)

            #Check if unavailable issue
            #If it's there, retry in 10 seconds
            while 'Service Unavailable' in soup.text:
                print 'No Server Avaialble.  Trying Again in 10 seconds...'
                time.sleep(10)
                web_html = urlopen(web_add).read()
                soup = BeautifulSoup(web_html)

            #Get the week and year of the predictions
            title= soup.find("title")
            web_week=int(title.text.split(' ')[1])

            #catch week 17 leaking into the start of the year
            if (month=='09') and (int(web_week)>10):
                print "Warning: Web week %s found in month 09" % web_week
            
            
            comments=soup.find_all(string=lambda text:isinstance(text,Comment))
            if 'FILE ARCHIVED' in soup.find_all(string=lambda text:isinstance(text,Comment))[-1]:            
                web_year = comments[-1].split(' ')[11]
                web_month = comments[-1].split(' ')[9]
                if web_year != year:
                    print "Warning: Web year and Season Year mismatch.  Web month is %s" % web_month

            else:
                print 'Error parsing year info - using previous query info' 
                #this is fine as long as it doesn't happen on the first query

            #Get the player rankings and injury status
            table= soup.find("table", { "id" : "data"} )

            #Skip the first row of the table, since it is header info
            rows=table.findAll('tr')
            iterrows = iter(rows)
            next(iterrows)

            #Iterate over reaming rows
            for tr in iterrows:

                #Find all columns and exctract info
                cols = tr.findAll('td')  

                #Skip empty rows and free agents (cols[4].find('a') empty means free agent)
                if cols[1].find('a').contents:
                    team = cols[1].find('span',{'class':'tiny'}).text.split(' ')[0][1:]   
                    player = cols[1].find('a').contents[0]

                    #If the player isn't in our database, find the closest matching name
                    #It's okay if it comes back with bull, since the player + team + season + week
                    #combo will prevent it from being entered
                    if player not in gameStats['Player'].tolist():
                        player = str(nfldb.player_search(db,player)[0])

                    rank=cols[0].text
                    best_rank=cols[2].text
                    worst_rank=cols[3].text
                    average_rank=cols[4].text

                    #Check for old format where they use an image
                    if cols[1].find('img'):
                        injury_status=cols[1].find('img').get('title')[0]         
                    #Check for the end of year format where they use a red letter instead of img
                    elif len(cols[1].findAll('span',{'class':'tiny'}))>1:
                        injury_status=cols[1].findAll('span',{'class':'tiny'})[1].text
                    else:
                        injury_status='H' #H for healthy

                    #Use player,team,week,and year to fill in the information in our dataframe        
                    selector=( (gameStats.Player==player) & (gameStats.Team ==team) &\
                               (gameStats.SeasonWeek==int(web_week)) & (gameStats.Year==float(year)) )


                    #If there is a player,team,season,and week match.. fill in the data
                    if len(gameStats[selector])>0: 
                        gameStats.ix[selector, 'Expert Rank'] = rank         
                        gameStats.ix[selector, 'Average Expert Rank'] = average_rank
                        gameStats.ix[selector, 'Best Expert Rank']=best_rank
                        gameStats.ix[selector, 'Worst Expert Rank']=worst_rank
                        gameStats.ix[selector, 'Injury Status']=injury_status
                        
        print 'Done with %s/%s' % (month,day)

Done with 09/3
Done with 09/6
Done with 09/9
Done with 09/12
Done with 09/15
Done with 09/18
Done with 09/21
Done with 09/24
Done with 09/27
Done with 09/30
Done with 10/3
Done with 10/6
Done with 10/9
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Done with 10/12
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Done with 10/15
Done with 10/18
Done with 10/21
Done with 10/24
Done with 10/27
Done with 10/30
Done with 11/3
Done with 11/6
Done with 11/9
Done with 11/12
Done with 11/15
Done with 11/18
Done with 11/21
Done with 11/24
Done with 11/27
Done with 11/30
Done with 12/3
Done with 12/6
Done with 12/9
Done with 12/12
Done with 12/15
Done with 12/18
Done with 12/21
Done with 12/24
Done with 12/27
Done with 12/30
Done wit

In [8]:
gameStats.to_csv('gameStats_withExperts.csv')

In [9]:
#PPR Version
year='2012'

#Season starts first week in September, ends at end of December
for month in ['09','10','11','12','01']:
    for day in range(3,31,3):
        for pos in ['wr','rb','te']:
            web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/ppr-%s.php' % (year,str(month).zfill(2),str(day).zfill(2),pos)

            #Check Jan of the next year too
            if month=='01':   
                web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/ppr-%s.php' % (str(int(year)+1),str(month).zfill(2),str(day).zfill(2),pos)
 
            web_html = urlopen(web_add).read()
            soup = BeautifulSoup(web_html)

            #Check if unavailable issue
            #If it's there, retry in 10 seconds
            while 'Service Unavailable' in soup.text:
                print 'No Server Avaialble.  Trying Again in 10 seconds...'
                time.sleep(10)
                web_html = urlopen(web_add).read()
                soup = BeautifulSoup(web_html)

            #Get the week and year of the predictions
            title= soup.find("title")
            web_week=int(title.text.split(' ')[1])
            
            #catch week 17 leaking into the start of the year
            if (month=='09') and (int(web_week)>10):
                print "Warning: Web week %s found in month 09" % web_week

            
            comments=soup.find_all(string=lambda text:isinstance(text,Comment))
            if 'FILE ARCHIVED' in soup.find_all(string=lambda text:isinstance(text,Comment))[-1]:            
                web_year = comments[-1].split(' ')[11]
                web_month = comments[-1].split(' ')[9]
                if web_year != year:
                    print "Warning: Web year and Season Year mismatch.  Web month is %s" % web_month

            else:
                print 'Error parsing year info - using previous query info' 
                #this is fine as long as it doesn't happen on the first query
              
                
            #Get the player rankings and injury status
            table= soup.find("table", { "id" : "data"} )

            #Skip the first row of the table, since it is header info
            rows=table.findAll('tr')
            iterrows = iter(rows)
            next(iterrows)

            #Iterate over reaming rows
            for tr in iterrows:

                #Find all columns and exctract info
                cols = tr.findAll('td')  

                #Skip empty rows and free agents (cols[4].find('a') empty means free agent)
                if cols[1].find('a').contents:
                    team = cols[1].find('span',{'class':'tiny'}).text.split(' ')[0][1:]   
                    player = cols[1].find('a').contents[0]

                    #If the player isn't in our database, find the closest matching name
                    #It's okay if it comes back with bull, since the player + team + season + week
                    #combo will prevent it from being entered
                    if player not in gameStats['Player'].tolist():
                        player = str(nfldb.player_search(db,player)[0])

                    rank=cols[0].text
                    best_rank=cols[2].text
                    worst_rank=cols[3].text
                    average_rank=cols[4].text

                    #Check for old format where they use an image
                    if cols[1].find('img'):
                        injury_status=cols[1].find('img').get('title')[0]         
                    #Check for the end of year format where they use a red letter instead of img
                    elif len(cols[1].findAll('span',{'class':'tiny'}))>1:
                        injury_status=cols[1].findAll('span',{'class':'tiny'})[1].text
                    else:
                        injury_status='H' #H for healthy

                    #Use player,team,week,and year to fill in the information in our dataframe        
                    selector=( (gameStats.Player==player) & (gameStats.Team ==team) &\
                               (gameStats.SeasonWeek==int(web_week)) & (gameStats.Year==float(year)) )


                    #If there is a player,team,season,and week match.. fill in the data
                    if len(gameStats[selector])>0: 
                        gameStats.ix[selector, 'PPR Expert Rank'] = rank         
                        gameStats.ix[selector, 'PPR Average Expert Rank'] = average_rank
                        gameStats.ix[selector, 'PPR Best Expert Rank']=best_rank
                        gameStats.ix[selector, 'PPR Worst Expert Rank']=worst_rank
                        gameStats.ix[selector, 'PPR Injury Status']=injury_status
                        
        print 'Done with %s/%s' % (month,day)

Done with 09/3
Done with 09/6
Done with 09/9
Done with 09/12
Done with 09/15
Done with 09/18
Done with 09/21
Done with 09/24
Done with 09/27
Done with 09/30
Done with 10/3
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Done with 10/6
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Done with 10/9
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Done with 10/12
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Done with 10/15
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Done with 10/18
Done with 10/21
Done with 10/24
Done with 10/27
Done with 10/30
Done with 

AttributeError: 'NoneType' object has no attribute 'text'

In [10]:
gameStats.to_csv('gameStats_withExperts.csv')

Skipping 2013 since it wasn't acrhived

In [11]:
year='2014'

#Season starts first week in September, ends at end of December
for month in ['09','10','11','12','01']:
    for day in range(3,31,3):
        for pos in ['wr','rb','te','qb']:
            web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/%s.php' % (year,str(month).zfill(2),str(day).zfill(2),pos)

            #Check Jan of the next year too
            if month=='01':   
                web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/%s.php' % (str(int(year)+1),str(month).zfill(2),str(day).zfill(2),pos)

            web_html = urlopen(web_add).read()
            soup = BeautifulSoup(web_html)

            #Check if unavailable issue
            #If it's there, retry in 10 seconds
            while 'Service Unavailable' in soup.text:
                print 'No Server Avaialble.  Trying Again in 10 seconds...'
                time.sleep(10)
                web_html = urlopen(web_add).read()
                soup = BeautifulSoup(web_html)

            #Get the week and year of the predictions
            title= soup.find("title")
            web_week=int(title.text.split(' ')[1])

            #catch week 17 leaking into the start of the year
            if (month=='09') and (int(web_week)>10):
                print "Warning: Web week %s found in month 09" % web_week
            

            comments=soup.find_all(string=lambda text:isinstance(text,Comment))
            if 'FILE ARCHIVED' in soup.find_all(string=lambda text:isinstance(text,Comment))[-1]:            
                web_year = comments[-1].split(' ')[11]
                web_month = comments[-1].split(' ')[9]
                if web_year != year:
                    print "Warning: Web year and Season Year mismatch.  Web month is %s" % web_month

            else:
                print 'Error parsing year info - using previous query info' 
                #this is fine as long as it doesn't happen on the first query


            #Get the player rankings and injury status
            table= soup.find("table", { "id" : "data"} )

            #Skip the first row of the table, since it is header info
            rows=table.findAll('tr')
            iterrows = iter(rows)
            next(iterrows)

            #Iterate over reaming rows
            for tr in iterrows:

                #Find all columns and exctract info
                cols = tr.findAll('td')  

                if len(cols)>1:

                    #Skip empty rows and free agents (cols[4].find('a') empty means free agent)
                    if cols[1].find('a').contents:
                        team = cols[1].find('small').text.split(')')[0].split('(')[-1]   
                        player = cols[1].find('a').contents[0]

                        #If the player isn't in our database, find the closest matching name
                        #It's okay if it comes back with bull, since the player + team + season + week
                        #combo will prevent it from being entered
                        if player not in gameStats['Player'].tolist():
                            player = str(nfldb.player_search(db,player)[0])

                        rank=cols[0].text
                        best_rank=cols[2].text
                        worst_rank=cols[3].text
                        average_rank=cols[4].text

                        #Check for injury info
                        if cols[1].find('div',{'class':'pull-right'}):
                            injury_status=cols[1].find('div',{'class':'pull-right'}).text[0]        
                        else:
                            injury_status='H' #H for healthy

                        #Use player,team,week,and year to fill in the information in our dataframe        
                        selector=( (gameStats.Player==player) & (gameStats.Team ==team) &\
                                   (gameStats.SeasonWeek==int(web_week)) & (gameStats.Year==float(year)) )


                        #If there is a player,team,season,and week match.. fill in the data
                        if len(gameStats[selector])>0: 
                            gameStats.ix[selector, 'Expert Rank'] = rank         
                            gameStats.ix[selector, 'Average Expert Rank'] = average_rank
                            gameStats.ix[selector, 'Best Expert Rank']=best_rank
                            gameStats.ix[selector, 'Worst Expert Rank']=worst_rank
                            gameStats.ix[selector, 'Injury Status']=injury_status
                        
        print 'Done with %s/%s' % (month,day)

Done with 09/3
Done with 09/6
Done with 09/9
Done with 09/12
Done with 09/15
Done with 09/18
Done with 09/21
Done with 09/24
Done with 09/27
Done with 09/30
Done with 10/3
Done with 10/6
Done with 10/9
Done with 10/12
Done with 10/15
Done with 10/18
Done with 10/21
Done with 10/24
Done with 10/27
Done with 10/30
Done with 11/3
Done with 11/6
Done with 11/9
Done with 11/12
Done with 11/15
Done with 11/18
Done with 11/21
Done with 11/24
Done with 11/27
Done with 11/30
Done with 12/3
Done with 12/6
Done with 12/9
Done with 12/12
Done with 12/15
Done with 12/18
Done with 12/21
Done with 12/24
Done with 12/27


ValueError: invalid literal for int() with base 10: 'WR'

In [12]:
gameStats.to_csv('gameStats_withExperts.csv')

In [13]:
#2014 basically doesn't exist..have to skip 09... PPR Version
year='2014'

#Season starts first week in September, ends at end of December
for month in ['10','11','12','01']:
    for day in range(3,31,3):
        for pos in ['wr','rb','te']:
            web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/ppr-%s.php' % (year,str(month).zfill(2),str(day).zfill(2),pos)

            #Check Jan of the next year too
            if month=='01':   
                web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/ppr-%s.php' % (str(int(year)+1),str(month).zfill(2),str(day).zfill(2),pos)

            web_html = urlopen(web_add).read()
            soup = BeautifulSoup(web_html)

            #Check if unavailable issue
            #If it's there, retry in 10 seconds
            while 'Service Unavailable' in soup.text:
                print 'No Server Avaialble.  Trying Again in 10 seconds...'
                time.sleep(10)
                web_html = urlopen(web_add).read()
                soup = BeautifulSoup(web_html)

            #Get the week and year of the predictions
            title= soup.find("title")
            web_week=int(title.text.split(' ')[1])

            #catch week 17 leaking into the start of the year
            if (month=='09') and (int(web_week)>10):
                print "Warning: Web week %s found in month 09" % web_week



            comments=soup.find_all(string=lambda text:isinstance(text,Comment))
            if 'FILE ARCHIVED' in soup.find_all(string=lambda text:isinstance(text,Comment))[-1]:            
                web_year = comments[-1].split(' ')[11]
                web_month = comments[-1].split(' ')[9]
                if web_year != year:
                    print "Warning: Web year and Season Year mismatch.  Web month is %s" % web_month

            else:
                print 'Error parsing year info - using previous query info' 
                #this is fine as long as it doesn't happen on the first query


            #Get the player rankings and injury status
            table= soup.find("table", { "id" : "data"} )

            #Skip the first row of the table, since it is header info
            rows=table.findAll('tr')
            iterrows = iter(rows)
            next(iterrows)

            #Iterate over reaming rows
            for tr in iterrows:

                #Find all columns and exctract info
                cols = tr.findAll('td')  

                if len(cols)>1:

                    #Skip empty rows and free agents (cols[4].find('a') empty means free agent)
                    if cols[1].find('a').contents:
                        team = cols[1].find('small').text.split(')')[0].split('(')[-1]   
                        player = cols[1].find('a').contents[0]

                        #If the player isn't in our database, find the closest matching name
                        #It's okay if it comes back with bull, since the player + team + season + week
                        #combo will prevent it from being entered
                        if player not in gameStats['Player'].tolist():
                            player = str(nfldb.player_search(db,player)[0])

                        rank=cols[0].text
                        best_rank=cols[2].text
                        worst_rank=cols[3].text
                        average_rank=cols[4].text

                        #Check for injury info
                        if cols[1].find('div',{'class':'pull-right'}):
                            injury_status=cols[1].find('div',{'class':'pull-right'}).text[0]        
                        else:
                            injury_status='H' #H for healthy

                        #Use player,team,week,and year to fill in the information in our dataframe        
                        selector=( (gameStats.Player==player) & (gameStats.Team ==team) &\
                                   (gameStats.SeasonWeek==int(web_week)) & (gameStats.Year==float(year)) )


                        #If there is a player,team,season,and week match.. fill in the data
                        if len(gameStats[selector])>0: 
                            gameStats.ix[selector, 'PPR Expert Rank'] = rank         
                            gameStats.ix[selector, 'PPR Average Expert Rank'] = average_rank
                            gameStats.ix[selector, 'PPR Best Expert Rank']=best_rank
                            gameStats.ix[selector, 'PPR Worst Expert Rank']=worst_rank
                            gameStats.ix[selector, 'PPR Injury Status']=injury_status
                            
        print 'Done with %s/%s' % (month,day)                            

Done with 10/3
Done with 10/6
Done with 10/9
Done with 10/12
Done with 10/15
Done with 10/18
Done with 10/21
Done with 10/24
Done with 10/27
Done with 10/30
Done with 11/3
Done with 11/6
Done with 11/9
Done with 11/12
Done with 11/15
Done with 11/18
Done with 11/21
Done with 11/24
Done with 11/27
Done with 11/30
Done with 12/3
Done with 12/6
Done with 12/9
Done with 12/12
Done with 12/15
Done with 12/18
Done with 12/21
Done with 12/24


ValueError: invalid literal for int() with base 10: 'WR'

In [14]:
gameStats.to_csv('gameStats_withExperts.csv')

In [55]:
year='2015'

#Season starts first week in September, ends at end of December
for month in ['09','10','11','12','01']:
    for day in range(3,31,3):
        for pos in ['wr','rb','te','qb']:
            web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/%s.php' % (year,str(month).zfill(2),str(day).zfill(2),pos)

            #Check Jan of the next year too
            if month=='01':   
                web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/%s.php' % (str(int(year)+1),str(month).zfill(2),str(day).zfill(2),pos)
 
            web_html = urlopen(web_add).read()
            soup = BeautifulSoup(web_html)

            #Check if unavailable issue
            #If it's there, retry in 10 seconds
            while 'Service Unavailable' in soup.text:
                print 'No Server Avaialble.  Trying Again in 10 seconds...'
                time.sleep(10)
                web_html = urlopen(web_add).read()
                soup = BeautifulSoup(web_html)

            #Get the week and year of the predictions
            title= soup.find("title")
            web_week=int(title.text.split(' ')[1])

            #catch week 17 leaking into the start of the year
            if (month=='09') and (int(web_week)>10):
                print "Warning: Web week %s found in month 09" % web_week
            
            
            comments=soup.find_all(string=lambda text:isinstance(text,Comment))
            if 'FILE ARCHIVED' in soup.find_all(string=lambda text:isinstance(text,Comment))[-1]:            
                web_year = comments[-1].split(' ')[11]
                web_month = comments[-1].split(' ')[9]
                if web_year != year:
                    print "Warning: Web year and Season Year mismatch.  Web month is %s" % web_month

            else:
                print 'Error parsing year info - using previous query info' 
                #this is fine as long as it doesn't happen on the first query



            #Get the player rankings and injury status
            table= soup.find("table", { "id" : "data"} )

            #Skip the first row of the table, since it is header info
            rows=table.findAll('tr')
            iterrows = iter(rows)
            next(iterrows)

            #Iterate over reaming rows
            for tr in iterrows:

                #Find all columns and exctract info
                cols = tr.findAll('td')  
                
                if len(cols)>1:

                    #Deal with the addition of the opponent column in second week
                    if len(cols)==6:
                        first_stat_col=2
                    elif len(cols)==7:
                        first_stat_col=3


                    #Skip empty rows and free agents (cols[4].find('a') empty means free agent)
                    if cols[1].find('a').contents:
                        team = cols[1].find('small').text.split(' ')[0] 
                        player = cols[1].find('a').contents[0]

                        #If the player isn't in our database, find the closest matching name
                        #It's okay if it comes back with bull, since the player + team + season + week
                        #combo will prevent it from being entered
                        if player not in gameStats['Player'].tolist():
                            player = str(nfldb.player_search(db,player)[0])

                        rank=cols[0].text
                        best_rank=cols[first_stat_col].text
                        worst_rank=cols[first_stat_col+1].text
                        average_rank=cols[first_stat_col+2].text

                        #Check for injury info in the original format of page
                        if cols[1].find('div',{'class':'pull-right'}):                        
                            if cols[1].find('div',{'class':'pull-right'}).text==u'\xa0':
                                injury_status='H'
                            else:
                                injury_status=cols[1].find('div',{'class':'pull-right'}).text[0] 
                        #Check updated format of the page
                        elif cols[1].find('small',{'class':'dl'}):                                                        
                            injury_status=cols[1].find('small',{'class':'dl'}).text[0]
                        else:
                            injury_status='H' #H for healthy


                        #Use player,team,week,and year to fill in the information in our dataframe        
                        selector=( (gameStats.Player==player) & (gameStats.Team ==team) &\
                                   (gameStats.SeasonWeek==int(web_week)) & (gameStats.Year==float(year)) )

                        #If there is a player,team,season,and week match.. fill in the data
                        if len(gameStats[selector])>0: 
                            gameStats.ix[selector, 'Expert Rank'] = rank         
                            gameStats.ix[selector, 'Average Expert Rank'] = average_rank
                            gameStats.ix[selector, 'Best Expert Rank']=best_rank
                            gameStats.ix[selector, 'Worst Expert Rank']=worst_rank
                            gameStats.ix[selector, 'Injury Status']=injury_status
                        
        print 'Done with %s/%s' % (month,day)

Done with 09/3
Done with 09/6
Done with 09/9
Done with 09/12
Done with 09/15
Done with 09/18
Done with 09/21
Done with 09/24
Done with 09/27
Done with 09/30
Done with 10/3
Done with 10/6
Done with 10/9
Done with 10/12
Done with 10/15
Done with 10/18
Done with 10/21
Done with 10/24
Done with 10/27
Done with 10/30
Done with 11/3
Done with 11/6
Done with 11/9
Done with 11/12
Done with 11/15
Done with 11/18
Done with 11/21
Done with 11/24
Done with 11/27
Error parsing year info - using previous query info
Done with 11/30
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Done with 12/3
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Error parsing year info - using previous query info
Done with 12/6
Error parsing year info - using

ValueError: invalid literal for int() with base 10: 'WR'

In [56]:
gameStats.to_csv('gameStats_withExperts.csv')

In [57]:
#PPR Version
year='2015'

#Season starts first week in September, ends at end of December
for month in ['09','10','11','12','01']:
    for day in range(3,31,3):
        for pos in ['wr','rb','te']:
            web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/ppr-%s.php' % (year,str(month).zfill(2),str(day).zfill(2),pos)

            #Check Jan of the next year too
            if month=='01':   
                web_add = 'https://web.archive.org/web/%s%s%s/http://www.fantasypros.com/nfl/rankings/ppr-%s.php' % (str(int(year)+1),str(month).zfill(2),str(day).zfill(2),pos)
 
            web_html = urlopen(web_add).read()
            soup = BeautifulSoup(web_html)

            #Check if unavailable issue
            #If it's there, retry in 10 seconds
            while 'Service Unavailable' in soup.text:
                print 'No Server Avaialble.  Trying Again in 10 seconds...'
                time.sleep(10)
                web_html = urlopen(web_add).read()
                soup = BeautifulSoup(web_html)

            #Get the week and year of the predictions
            title= soup.find("title")
            web_week=int(title.text.split(' ')[1])

            #catch week 17 leaking into the start of the year
            if (month=='09') and (int(web_week)>10):
                print "Warning: Web week %s found in month 09" % web_week
            
            
            comments=soup.find_all(string=lambda text:isinstance(text,Comment))
            if 'FILE ARCHIVED' in soup.find_all(string=lambda text:isinstance(text,Comment))[-1]:            
                web_year = comments[-1].split(' ')[11]
                web_month = comments[-1].split(' ')[9]
                if web_year != year:
                    print "Warning: Web year and Season Year mismatch.  Web month is %s" % web_month

            else:
                print 'Error parsing year info - using previous query info' 
                #this is fine as long as it doesn't happen on the first query


            #Get the player rankings and injury status
            table= soup.find("table", { "id" : "data"} )

            #Skip the first row of the table, since it is header info
            rows=table.findAll('tr')
            iterrows = iter(rows)
            next(iterrows)

            #Iterate over reaming rows
            for tr in iterrows:

                #Find all columns and exctract info
                cols = tr.findAll('td')  
                
                if len(cols)>1:

                    #Deal with the addition of the opponent column in second week
                    if len(cols)==6:
                        first_stat_col=2
                    elif len(cols)==7:
                        first_stat_col=3


                    #Skip empty rows and free agents (cols[4].find('a') empty means free agent)
                    if cols[1].find('a').contents:
                        team = cols[1].find('small').text.split(' ')[0] 
                        player = cols[1].find('a').contents[0]

                        #If the player isn't in our database, find the closest matching name
                        #It's okay if it comes back with bull, since the player + team + season + week
                        #combo will prevent it from being entered
                        if player not in gameStats['Player'].tolist():
                            player = str(nfldb.player_search(db,player)[0])

                        rank=cols[0].text
                        best_rank=cols[first_stat_col].text
                        worst_rank=cols[first_stat_col+1].text
                        average_rank=cols[first_stat_col+2].text

                        #Check for injury info in the original format of page
                        if cols[1].find('div',{'class':'pull-right'}):                        
                            if cols[1].find('div',{'class':'pull-right'}).text==u'\xa0':
                                injury_status='H'
                            else:
                                injury_status=cols[1].find('div',{'class':'pull-right'}).text[0] 
                        #Check updated format of the page
                        elif cols[1].find('small',{'class':'dl'}):                                                        
                            injury_status=cols[1].find('small',{'class':'dl'}).text[0]
                        else:
                            injury_status='H' #H for healthy


                          #Use player,team,week,and year to fill in the information in our dataframe        
                        selector=( (gameStats.Player==player) & (gameStats.Team ==team) &\
                                   (gameStats.SeasonWeek==int(web_week)) & (gameStats.Year==float(year)) )

                        #If there is a player,team,season,and week match.. fill in the data
                        if len(gameStats[selector])>0: 
                            gameStats.ix[selector, 'PPR Expert Rank'] = rank         
                            gameStats.ix[selector, 'PPR Average Expert Rank'] = average_rank
                            gameStats.ix[selector, 'PPR Best Expert Rank']=best_rank
                            gameStats.ix[selector, 'PPR Worst Expert Rank']=worst_rank
                            gameStats.ix[selector, 'PPR Injury Status']=injury_status
                        
        print 'Done with %s/%s' % (month,day)

Done with 09/3
Done with 09/6
Done with 09/9
Done with 09/12
Done with 09/15
Done with 09/18
Done with 09/21
Done with 09/24
Done with 09/27
Done with 09/30
Done with 10/3
Done with 10/6
Done with 10/9
Done with 10/12
Done with 10/15
Done with 10/18
Done with 10/21
Done with 10/24
Done with 10/27
Done with 10/30
Done with 11/3
Done with 11/6
Done with 11/9
Done with 11/12
Done with 11/15
Done with 11/18
Done with 11/21
Done with 11/24
Done with 11/27
Done with 11/30
Done with 12/3
Done with 12/6
Done with 12/9
Done with 12/12
Done with 12/15
Done with 12/18
Done with 12/21
Done with 12/24
Done with 12/27
Done with 12/30
Done with 01/3


ValueError: invalid literal for int() with base 10: 'TE'

In [58]:
gameStats.to_csv('gameStats_withExperts.csv')